# YOLO File Format
https://github.com/AlexeyAB/Yolo_mark/issues/60
Described above and a royal pain in the ass

# imports

In [44]:
import pandas as pd
import numpy as np
from PIL import Image
from operator import itemgetter
import shutil
import os
from ast import literal_eval


In [18]:
source_folder = './airbus'

In [48]:

df = pd.read_csv(source_folder+'/annotations.csv', converters={"geometry": literal_eval})
df

,id,image_id,geometry,class
0,1,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(135, 522), (245, 522), (245, 600), (135, 600...",Airplane
1,2,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1025, 284), (1125, 284), (1125, 384), (1025,...",Airplane
2,3,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1058, 1503), (1130, 1503), (1130, 1568), (10...",Airplane
3,4,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(813, 1518), (885, 1518), (885, 1604), (813, ...",Airplane
4,5,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(594, 938), (657, 938), (657, 1012), (594, 10...",Airplane
...,...,...,...,...
3420,3421,48ef8e15-a43c-406b-9d3c-e815164b96d1.jpg,"[(1059, 393), (1187, 393), (1187, 522), (1059,...",Airplane
3421,3422,48ef8e15-a43c-406b-9d3c-e815164b96d1.jpg,"[(932, 355), (1028, 355), (1028, 461), (932, 4...",Airplane
3422,3423,48ef8e15-a43c-406b-9d3c-e815164b96d1.jpg,"[(2461, 244), (2560, 244), (2560, 345), (2461,...",Truncated_airplane
3423,3424,48ef8e15-a43c-406b-9d3c-e815164b96d1.jpg,"[(1820, 344), (1937, 344), (1937, 443), (1820,...",Airplane


In [65]:
# expects csv in same form as this dataset https://www.kaggle.com/airbusgeo/airbus-aircrafts-sample-dataset?select=annotations.csv
# id, image_name, geometry, class
classes = df['class'].unique()
print("using classes", classes)

# class map i.e. {class name: class index, etc.}
class_map = {clz:clz_index for clz_index, clz in enumerate(classes)}

print('class map', class_map)

images = df['image_id'].unique()
print(len(images), "images detected")

using classes ['Airplane' 'Truncated_airplane']
class map {'Airplane': 0, 'Truncated_airplane': 1}
103 images detected


In [21]:

!cd airbus; ls; mkdir output_images; ls;

LICENSE.txt     annotations.csv images
README.md       extras          output_images
mkdir: output_images: File exists
LICENSE.txt     annotations.csv images
README.md       extras          output_images


In [63]:
# for an array of tuples find minimum and maximum of x and y
def getMinMaxOfXY(bbox):
    # item getter assumes an iterable and lets you use min/max over a tuple
    return {
        'maxX': max(bbox, key=itemgetter(0))[0],
        'maxY': max(bbox, key=itemgetter(1))[1],
        'minX': min(bbox, key=itemgetter(0))[0],
        'minY': min(bbox, key=itemgetter(1))[1]
    }

def getCentroidOfBbox(x1, x2, y1, y2):
    xCenter = (x1 + x2) / 2
    yCenter = (y1 + y2) / 2
    return (xCenter, yCenter)

def generateYoloTxtsForImage(image_name, objects):
    """
    objects: [{
        'geometry': Array(Tuple),
        'class': String
    }]
    """
    # open image and get width and height then close
    image_path = source_folder + '/images/' + image_name
    im = Image.open(image_path)
    w, h =  im.size
    im.close()

    yolo_objects = []

    # loop over each object in array of objects
    for object in objects:
        # geometry is the bounding box of the detected object
        bbox = object['geometry']

        # get min and max of x and y coordinates of bbox
        min_maxes_dict = getMinMaxOfXY(bbox)

        # height and width of detected object
        object_width = min_maxes_dict['maxX'] - min_maxes_dict['minX']
        object_height = min_maxes_dict['maxY'] - min_maxes_dict['minY']

        # get center of bbox
        centroid = getCentroidOfBbox(**{
            'x1': min_maxes_dict['maxX'],
            'x2': min_maxes_dict['minX'],
            'y1': min_maxes_dict['maxY'],
            'y2': min_maxes_dict['minY']
        })
        x_center = centroid[0]
        y_center = centroid[1]

        # get object name and respective 'class' (index of object name)
        object_name = object['class']
        object_class = class_map[object_name]


        # normalised bounding box width, height and centroid to be between 0 and 1.0 based on width and height of image
        normalised_width = object_width / w
        normalised_height = object_height / h
        normalised_x_center = x_center / w
        normalised_y_center = y_center / h

        yolo_objects.append({
            'object_class': object_class,
            'x': normalised_x_center,
            'y': normalised_y_center,
            'width': normalised_width,
            'height': normalised_height
        })

    # copy image from images to output_images
    # print("copying from", image_path)
    # print("to", source_folder+'/output_images/')
    shutil.copy(image_path, source_folder+'/output_images/')
    # get filename of image without extension or path
    filename = os.path.splitext(image_path)[0].split('/')[-1]
    # array of strings in format
    # <object-class> <x> <y> <width> <height>
    lines = ["{} {} {} {} {}".format(object['object_class'], object['x'], object['y'], object['width'], object['height']) for object in yolo_objects]
    # print(lines)
    # print("lines length", len(lines))

    txt_file = source_folder +'/output_images/' + filename + '.txt'
    # print("txt file", txt_file)

    # create txt with same filename as jpg and write lines to txt
    with open(txt_file, 'w') as f:
        f.write('\n'.join(lines))
    

for image_name in images:
    """
    create objects array like so
    objects: [{
        'geometry': Array(Tuple),
        'class': String
    }]
    """
    image_data = df.loc[df['image_id'] == image_name]
    # print(image_data)
    geometries = image_data['geometry']
    classes = image_data['class']

    n_rows = len(image_data.index)

    objects = []
    for i in range(n_rows):
        objects.append({
            'geometry': geometries.iloc[i],
            'class': classes.iloc[i] 
        })

    # print("objects is", objects)
    
    generateYoloTxtsForImage(image_name, objects)
    

    id                                  image_id  \
0    1  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
1    2  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
2    3  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
3    4  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
4    5  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
5    6  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
6    7  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
7    8  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
8    9  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
9   10  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
10  11  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
11  12  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
12  13  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
13  14  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
14  15  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
15  16  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
16  17  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   
17  18  4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg   

           